In [1]:
import pyBeamSim
import numpy as np
import pandas as pd

In [2]:
import torch
import torch.nn as nn

In [10]:
"""" variable dataframe created"""
vary = ['drift1_len', 'quad1_len', 'quad1_gra', 'drift2_len', 'quad2_len', 'quad2_gra']
low = [0.1, 0.1, -20, 0.1, 0.1, -20]
high = [0.5, 0.5, 20, 0.5, 0.5, 20]
units = ['m', 'm', 'T/m', 'm', 'm', 'T/m']
variable = pd.DataFrame(index=vary)
variable['low'] = low
variable['high'] = high
variable['units'] = units
variable

,low,high,units
drift1_len,0.1,0.5,m
quad1_len,0.1,0.5,m
quad1_gra,-20.0,20.0,T/m
drift2_len,0.1,0.5,m
quad2_len,0.1,0.5,m
quad2_gra,-20.0,20.0,T/m


In [3]:
data = {
        'q1' : np.random.rand(1000),
        'q2' :  np.random.rand(1000),
        'q3' : np.random.rand(1000),
        'q4' : np.random.rand(1000)
}
df = pd.DataFrame(data)
df

,q1,q2,q3,q4
0,0.839125,0.602589,0.702111,0.019946
1,0.944647,0.862822,0.438426,0.617143
2,0.944632,0.446936,0.463139,0.949192
3,0.308776,0.494719,0.051069,0.068664
4,0.607214,0.309880,0.760825,0.573454
...,...,...,...,...
995,0.018072,0.722773,0.337488,0.164093
996,0.376216,0.654177,0.896220,0.688942
997,0.589349,0.251092,0.021792,0.959933
998,0.690628,0.971425,0.847366,0.946027


In [4]:
df['x_avg_1'] = df['q1'] + df['q2']+np.random.normal(0, 1, size=(1000, ))
df['x_avg_2'] = df['q2'] + df['q3']+np.random.normal(0, 1, size=(1000, ))
df['x_avg_3'] = df['q1'] + df['q4']+np.random.normal(0, 1, size=(1000, ))
df['x_avg_4'] = df['q4'] - df['q3']+np.random.normal(0, 1, size=(1000, ))


In [5]:
df

,q1,q2,q3,q4,x_avg_1,x_avg_2,x_avg_3,x_avg_4
0,0.839125,0.602589,0.702111,0.019946,0.997892,3.515770,0.516437,-1.391773
1,0.944647,0.862822,0.438426,0.617143,2.209273,0.746107,0.586802,-1.195615
2,0.944632,0.446936,0.463139,0.949192,1.653066,0.476021,-0.196526,0.162144
3,0.308776,0.494719,0.051069,0.068664,0.391196,0.844749,2.806434,0.067730
4,0.607214,0.309880,0.760825,0.573454,2.219280,-0.900799,1.888970,-2.237474
...,...,...,...,...,...,...,...,...
995,0.018072,0.722773,0.337488,0.164093,-1.117398,0.465706,0.651894,0.374917
996,0.376216,0.654177,0.896220,0.688942,2.058256,2.129436,1.437618,2.193371
997,0.589349,0.251092,0.021792,0.959933,1.393356,-0.261288,1.242120,-0.176046
998,0.690628,0.971425,0.847366,0.946027,1.008358,1.316556,2.381614,0.559359


In [20]:
class EncoderLstm(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderLstm, self).__init__()
        self.hidden_size = hidden_size
        # self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)

    def forward(self, x, hidden):
        # embedded = self.embedding(x).view(1, 1, -1)
        output, hidden = self.lstm(x, hidden)
        return output, hidden

    def initHidden(self, batch_size):
        return (torch.zeros(1, batch_size, self.hidden_size),
                torch.zeros(1, batch_size, self.hidden_size))
# output_encoder shape: [batch_size, seq_len, hidden_size]
# hidden_encoder shape : [1, batch_size, hidden_size]

In [21]:
class DecoderLstm(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(DecoderLstm, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        # self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)

        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        # embedded = self.embedding(x).view(1, 1, -1)

        # x.shape~X[:, -1, :]: [batch_size, input_size]
        output, hidden = self.lstm(x.unsqueeze(1), hidden)
        # output_shape: [batch_size, seq_len=1, hidden_size]
        # hidden shape: [1, batch_size, hidden_size]
        output = self.fc(output.squeeze(1))
        # output_decoder  shape: [batch_size, output_size]
        return output, hidden

    def initHidden(self, batch_size):
        return (torch.zeros(1, batch_size, self.hidden_size),
                torch.zeros(1, batch_size, self.hidden_size))


### test

In [22]:
batch_size, seq_len, input_size = 10, 20, 30
hidden_size = 40
a = torch.ones(size=(batch_size, seq_len, input_size))  # (batch_size, input_size, seq_len)
ml = nn.LSTM(input_size, hidden_size, batch_first=True)
output, (hn, cn) = ml(a)

In [23]:
output.shape


torch.Size([10, 20, 40])

In [24]:
cn.shape

torch.Size([1, 10, 40])

### do

In [25]:
input_size = 1
hidden_size = 30
output_size = 1
batch_size = 1000
output_len = 4
#seq_len = 4

encoder = EncoderLstm(input_size, hidden_size)
decoder = DecoderLstm(input_size= input_size, hidden_size= hidden_size, output_size=output_size)

# X = torch.Tensor(df.iloc[:, :4])
# y = torch.Tensor(df.iloc[:, 4:])
X = df.iloc[:, :4]
y = df.iloc[:, 4:]
X.shape


(1000, 4)

In [26]:
X = torch.Tensor(X.values)
y = torch.Tensor(y.values)
X.shape

torch.Size([1000, 4])

In [27]:
X = X.reshape(X.size(0), X.size(1), -1)
y = y.reshape(y.size(0), y.size(1), -1)
y.shape


torch.Size([1000, 4, 1])

In [28]:
hidden_state = encoder.initHidden(batch_size)
# output, hidden = encoder(X)
hidden_state[0].shape

torch.Size([1, 1000, 30])

In [29]:
encoder_outputs, hidden_encoder = encoder(X, hidden_state)

In [30]:
encoder_outputs.shape

torch.Size([1000, 4, 30])

In [31]:
hidden_encoder[0].shape

torch.Size([1, 1000, 30])

take output_encoder as input_decoder

In [32]:
decoder_outputs, hidden_decoder = decoder(encoder_outputs[:, -1, :], hidden_encoder)

RuntimeError: input.size(-1) must be equal to input_size. Expected 1, got 30

In [ ]:
encoder_outputs[:, -1, :].shape

take input_encoder[:, -1, :] as input_decoder

In [ ]:
input_decoder = X[:, -1, :].unsqueeze(1)
output_total = []
for i in range(4):
    output_decoder, hidden_decoder = decoder(input_decoder, hidden_encoder)
    input_decoder = output_decoder.unsqueeze(1)
    hidden_encoder = hidden_decoder
    output_total.append(output_decoder)

In [ ]:
output_total[0].shape

In [ ]:
c = torch.concat(output_total, dim=1)
c

In [33]:
from model import Model

In [47]:
model = Model(input_size=1, hidden_size=30, output_size=1, output_len=4)

In [48]:
ina = X[:, -1, :]
ina.shape

torch.Size([1000, 1])

In [49]:
X.shape

torch.Size([1000, 4, 1])

In [50]:
result = model(X)

0
1
2
3


In [53]:
result.shape

torch.Size([1000, 4])